<a href="https://colab.research.google.com/github/ceccode/mlbookcamp/blob/main/Homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/06-trees/homework.md

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import sklearn

Load data

In [3]:
usecols = [
  'neighbourhood_group', 'room_type', 'latitude', 'longitude',
  'price',
  'minimum_nights',
  'number_of_reviews',
  'reviews_per_month',
  'calculated_host_listings_count',
  'availability_365']
 
data = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv', usecols=usecols)

In [4]:
print(data.shape)

(48895, 10)


In [5]:
data.head(10)

,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
1,Manhattan,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
2,Manhattan,40.80902,-73.94190,Private room,150,3,0,NaN,1,365
3,Brooklyn,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
4,Manhattan,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0
5,Manhattan,40.74767,-73.97500,Entire home/apt,200,3,74,0.59,1,129
6,Brooklyn,40.68688,-73.95596,Private room,60,45,49,0.40,1,0
7,Manhattan,40.76489,-73.98493,Private room,79,2,430,3.47,1,220
8,Manhattan,40.80178,-73.96723,Private room,79,2,118,0.99,1,0
9,Manhattan,40.71344,-73.99037,Entire home/apt,150,1,160,1.33,4,188


In [6]:
df = data.fillna(0)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_group             48895 non-null  object 
 1   latitude                        48895 non-null  float64
 2   longitude                       48895 non-null  float64
 3   room_type                       48895 non-null  object 
 4   price                           48895 non-null  int64  
 5   minimum_nights                  48895 non-null  int64  
 6   number_of_reviews               48895 non-null  int64  
 7   reviews_per_month               48895 non-null  float64
 8   calculated_host_listings_count  48895 non-null  int64  
 9   availability_365                48895 non-null  int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 3.7+ MB


train, val, test set will be 60%, 20%, 20% of the dataset respectively


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [10]:
len(df_train), len(df_val), len(df_test)

(29337, 9779, 9779)

In [11]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

del df_train['price']
del df_val['price']
del df_test['price']

Decision Tree

In [12]:

from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_text

In [13]:
train_dicts = df_train.fillna(0).to_dict(orient='records')

In [14]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

In [15]:
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=1,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [16]:
val_dicts = df_val.fillna(0).to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [17]:
print("The model training score is" , dt.score(X_train, y_train))

The model training score is 0.06375456851318562


In [18]:
# Then you can print the top 5 features in descending order of importance:

for importance, name in sorted(zip(dt.feature_importances_, df_train.columns),reverse=True)[:5]:
    print (name, importance)

room_type 0.0
reviews_per_month 0.0
number_of_reviews 0.0
neighbourhood_group 0.0
minimum_nights 0.0


In [19]:
print(export_text(dt, feature_names=dv.get_feature_names()))

|--- room_type=Entire home/apt <= 0.50
|   |--- value: [89.10]
|--- room_type=Entire home/apt >  0.50
|   |--- value: [211.78]



In [20]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
y_train_b = (np.log1p(y_train) >= np.log1p(152)).astype(int)
y_val_b = (np.log1p(y_val) >= np.log1p(152)).astype(int)
y_test_b = (np.log1p(y_val) >= np.log1p(152)).astype(int)

rf = RandomForestClassifier(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train_b)

y_pred = rf.predict_proba(X_val)[:, 1]
score_rf = roc_auc_score(y_val_b, y_pred)

score_rf


0.8756060048516114

In [25]:
# Evaluating the Algorithm

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_val_b, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_val_b, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val_b, y_pred)))

Mean Absolute Error: 0.23092340730136005
Mean Squared Error: 0.12816443399120567
Root Mean Squared Error: 0.3580006061324557


In [39]:
scores = []

for n in [50, 70, 120]:
    rf = RandomForestClassifier(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train_b)

    y_pred = rf.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val_b, y_pred)
    
    print(n, auc)
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val_b, y_pred)))
    scores.append((n, auc))


50 0.8923617437617177
Root Mean Squared Error: 0.34583059441923164
70 0.8933876053276424
Root Mean Squared Error: 0.34500406169949305
120 0.8958900237896477
Root Mean Squared Error: 0.3431236362302584


In [37]:
scores_2 = []

n_estimators = np.arange(10, 210, 10);

for n in n_estimators:

  scores_md = []

  for md in [10, 15, 20, 25]:
    rf = RandomForestClassifier(n_estimators=n, random_state=1, n_jobs=-1, max_depth=md)
    rf.fit(X_train, y_train_b)

    y_pred = rf.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val_b, y_pred)
       
    scores_md.append((n, auc))

  scores_2.append(scores_md)

scores_2

[[(10, 0.8925139272448827),
  (10, 0.8933937829747421),
  (10, 0.8858893807189134),
  (10, 0.8764001426267374)],
 [(20, 0.8959060261285197),
  (20, 0.8974466668095713),
  (20, 0.8945349110002011),
  (20, 0.88700909786306)],
 [(30, 0.8966332021786602),
  (30, 0.8993004819457892),
  (30, 0.8969182919088603),
  (30, 0.890128040543625)],
 [(40, 0.8969196812592118),
  (40, 0.9007656511309509),
  (40, 0.8986388038407994),
  (40, 0.89195654965558)],
 [(50, 0.8971868334839788),
  (50, 0.9010386088554017),
  (50, 0.8992927908991994),
  (50, 0.8933610588119943)],
 [(60, 0.8973693097667755),
  (60, 0.9013126334024435),
  (60, 0.8991690394785887),
  (60, 0.8943233575993296)],
 [(70, 0.897716969882457),
  (70, 0.901197565421531),
  (70, 0.8991141849495249),
  (70, 0.8947052800870765)],
 [(80, 0.897729052268551),
  (80, 0.9015791405716898),
  (80, 0.8993748618092596),
  (80, 0.8949861025269106)],
 [(90, 0.8977986190254462),
  (90, 0.9017838712699416),
  (90, 0.8995269956727691),
  (90, 0.89528781484

In [45]:
rf_features_importances = rf.feature_importances_
rf_features_importances

array([0.09280325, 0.04117641, 0.17403955, 0.19863317, 0.0619368 ,
       0.00154819, 0.00956486, 0.02416531, 0.00893816, 0.00066027,
       0.07397488, 0.08856502, 0.12760009, 0.0909841 , 0.00540995])

In [46]:
# Then you can print the top 5 features in descending order of importance:

for importance, name in sorted(zip(rf_features_importances, df_train.columns),reverse=True)[:5]:
    print (name, importance)

room_type 0.1986331671203065
longitude 0.1740395549510062
neighbourhood_group 0.09280324941482974
minimum_nights 0.06193680066632355
latitude 0.0411764078454028


In [51]:
!pip install xgboost

In [52]:
import xgboost as xgb

In [53]:
features = dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=y_train_b, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val_b, feature_names=features)

In [54]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=10)

In [56]:
y_pred = model.predict(dval)
roc_auc_score(y_val_b, y_pred)

0.8966331029393493

In [57]:
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [58]:
%%capture output

xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=200,
                  verbose_eval=5,
                  evals=watchlist)

In [59]:
s = output.stdout

In [60]:
print(s[:200])

[0]	train-auc:0.887824	val-auc:0.882088
[5]	train-auc:0.903108	val-auc:0.893592
[10]	train-auc:0.909514	val-auc:0.896969
[15]	train-auc:0.914989	val-auc:0.899269
[20]	train-auc:0.919179	val-auc:0.9008


In [62]:
%%capture output

xgb_params = {
    'eta': 0.1, 
    'max_depth': 10,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=200,
                  verbose_eval=5,
                  evals=watchlist)

In [63]:
y_pred = model.predict(dval)
roc_auc_score(y_val_b, y_pred)

0.8998353818311875

In [66]:
%%capture output

xgb_params = {
    'eta': 0.01, 
    'max_depth': 10,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=200,
                  verbose_eval=5,
                  evals=watchlist)


In [67]:

y_pred = model.predict(dval)
roc_auc_score(y_val_b, y_pred)

0.8990412688658892